In [18]:
import os
import pandas as pd

from galaxy_datasets.shared import label_metadata

In [19]:
df = pd.read_csv('data/gz3d_and_gz_desi_matches.csv')

In [20]:
#  '/share/nas2/walml/galaxy_zoo/decals/dr8/fits/'
df['galahad_fits_loc'] = df['brickid'].astype(str) + '/' + df['brickid'].astype(str) + '_' + df['objid'].astype(str) + '.fits'


#  '/share/nas2/walml/galaxy_zoo/decals/dr8/jpg/'
df['galahad_jpg_loc'] = df['brickid'].astype(str) + '/' + df['brickid'].astype(str) + '_' + df['objid'].astype(str) + '.jpg'

df['galahad_fits_loc'][0], df['galahad_jpg_loc'][0]

('593252/593252_2670.fits', '593252/593252_2670.jpg')

In [21]:
# df['dr8_id'].value_counts()
# TODO drop duplicate matches, temporarily
df = df.drop_duplicates(subset='dr8_id', keep=False)

In [22]:
# with open('data/galahad_fits_to_copy.txt', 'w') as f:
#     f.write('\n'.join(df['galahad_fits_loc']))

# with open('data/galahad_jpg_to_copy.txt', 'w') as f:
#     f.write('\n'.join(df['galahad_jpg_loc']))

In [23]:
# rsync --files-from data/galahad_fits_to_copy.txt -e 'ssh -A -J walml@external.jb.man.ac.uk' walml@galahad.ast.man.ac.uk:/share/nas2/walml/galaxy_zoo/decals/dr8/fits data/gz_desi/fits

In [24]:
# rsync --files-from data/galahad_jpg_to_copy.txt -e 'ssh -A -J walml@external.jb.man.ac.uk' walml@galahad.ast.man.ac.uk:/share/nas2/walml/galaxy_zoo/decals/dr8/jpg data/gz_desi/jpg

In [25]:
df['relative_desi_fits_loc'] = 'data/desi/fits/' + df['brickid'].astype(str) + '/' + df['brickid'].astype(str) + '_' + df['objid'].astype(str) + '.fits'
df['relative_desi_jpg_loc'] = 'data/desi/jpg/' + df['brickid'].astype(str) + '/' + df['brickid'].astype(str) + '_' + df['objid'].astype(str) + '.jpg'

In [26]:
df['relative_segmap_json_loc'] = df['relative_gz3d_fits_loc'].str.replace('/fits_gz/', '/segmaps/marks/', regex=False).str.replace('.fits.gz', '.json', regex=False)

In [27]:
df['relative_spiral_mask_loc'] = df['relative_gz3d_fits_loc'].apply(lambda x: x.replace('/fits_gz/', '/segmaps/masks/').replace('.fits.gz', '_spiral.jpg'))
df['relative_bar_mask_loc'] = df['relative_gz3d_fits_loc'].apply(lambda x: x.replace('/fits_gz/', '/segmaps/masks/').replace('.fits.gz', '_bar.jpg'))

In [28]:
df['relative_spiral_mask_loc'][0]

'data/gz3d/segmaps/masks/gz3d_1-379196_127_14728482_spiral.jpg'

In [29]:
len(df)

28983

In [30]:
df = df[df['relative_gz3d_fits_loc'].apply(lambda x: os.path.isfile(x))]
len(df)

28983

In [31]:
# add GZ DESI votes themselves




desi_vote_locs = [
    '/Volumes/beta/galaxy_zoo/decals/dr8/catalogs/training_catalogs/dr12_ortho_v5_labelled_catalog.parquet',
    '/Volumes/beta/galaxy_zoo/decals/dr8/catalogs/training_catalogs/dr5_ortho_v5_labelled_catalog.parquet',
    '/Volumes/beta/galaxy_zoo/decals/dr8/catalogs/training_catalogs/dr8_ortho_v5_labelled_catalog.parquet',
]

columns = [
    'dr8_id',
] + label_metadata.decals_all_campaigns_ortho_label_cols
gz_desi_votes = pd.concat([pd.read_parquet(loc, columns=columns) for loc in desi_vote_locs])

df = pd.merge(df, gz_desi_votes, on='dr8_id', validate='one_to_one', how='outer')
# left, because some have no votes (about half, 14k of 29k have votes)
# now outer, because we want to learn to classify as usual
df[label_metadata.decals_all_campaigns_ortho_label_cols] = df[label_metadata.decals_all_campaigns_ortho_label_cols].fillna(0)



FileNotFoundError: [Errno 2] No such file or directory: '/Volumes/beta/galaxy_zoo/decals/dr8/catalogs/training_catalogs/dr12_ortho_v5_labelled_catalog.parquet'

In [ ]:
# add GZ DESI vote predictions (just for filtering)

columns = [
    'dr8_id',
    'smooth-or-featured_featured-or-disk_fraction',
    'disk-edge-on_yes_fraction',
    'has-spiral-arms_yes_fraction',
    'spiral-arm-count_2_fraction'
]
gz_desi_preds = pd.read_parquet(
    '/Users/user/repos/desi-predictions/results/final/gz_desi_deep_learning_catalog_advanced.parquet',
    columns=columns
)

In [ ]:
gz_desi_preds['brickid'] = gz_desi_preds['dr8_id'].apply(lambda x: x.split('_')[0])
gz_desi_preds['objid'] = gz_desi_preds['dr8_id'].apply(lambda x: x.split('_')[1])


In [ ]:
del df['brickid']
del df['objid']
df = pd.merge(df, gz_desi_preds, on='dr8_id', validate='one_to_one', how='inner')

In [ ]:
df['smooth-or-featured_featured-or-disk_fraction'].isna().sum(), df['brickid'].isna().sum()

(0, 0)

In [ ]:
len(df)

415591

In [ ]:
df['galahad_jpg_loc'] = '/share/nas2/walml/galaxy_zoo/decals/dr8/jpg/' + df['brickid'].astype(str) + '/' + df['brickid'].astype(str) + '_' + df['objid'].astype(str) + '.jpg'
df['galahad_jpg_loc'].iloc[0]

'/share/nas2/walml/galaxy_zoo/decals/dr8/jpg/593252/593252_2670.jpg'

In [ ]:
df['relative_spiral_mask_loc'] = df['relative_spiral_mask_loc'].fillna('')
df['relative_bar_mask_loc'] = df['relative_bar_mask_loc'].fillna('')

In [ ]:
((df['relative_spiral_mask_loc'].str.startswith('data/gz3d/segmaps/masks/')) | (df['relative_spiral_mask_loc'] == '')).mean()

1.0

In [ ]:
df['relative_spiral_mask_loc'] = df['relative_spiral_mask_loc'].astype(str)
df['relative_bar_mask_loc'] = df['relative_bar_mask_loc'].astype(str)

In [ ]:
df.to_parquet('data/gz3d_and_gz_desi_master_catalog.parquet', index=False)

In [ ]:
df = pd.read_parquet('data/gz3d_and_gz_desi_master_catalog.parquet')

In [ ]:
# merge in gz2 matches on dr8_id
gz2_desi_matches = pd.read_csv('/Users/user/repos/zoobot-3d/data/gz2_and_gz_desi_matches.csv')
gz2_desi_matches.columns.values


array(['dr7objid', 'ra_gz2', 'dec_gz2', 'smooth-or-featured-gz2_smooth',
       'smooth-or-featured-gz2_featured-or-disk',
       'smooth-or-featured-gz2_artifact', 'disk-edge-on-gz2_yes',
       'disk-edge-on-gz2_no', 'bar-gz2_yes', 'bar-gz2_no',
       'has-spiral-arms-gz2_yes', 'has-spiral-arms-gz2_no',
       'bulge-size-gz2_no', 'bulge-size-gz2_just-noticeable',
       'bulge-size-gz2_obvious', 'bulge-size-gz2_dominant',
       'something-odd-gz2_yes', 'something-odd-gz2_no',
       'how-rounded-gz2_round', 'how-rounded-gz2_in-between',
       'how-rounded-gz2_cigar', 'bulge-shape-gz2_round',
       'bulge-shape-gz2_boxy', 'bulge-shape-gz2_no-bulge',
       'spiral-winding-gz2_tight', 'spiral-winding-gz2_medium',
       'spiral-winding-gz2_loose', 'spiral-arm-count-gz2_1',
       'spiral-arm-count-gz2_2', 'spiral-arm-count-gz2_3',
       'spiral-arm-count-gz2_4', 'spiral-arm-count-gz2_more-than-4',
       'spiral-arm-count-gz2_cant-tell',
       'smooth-or-featured-gz2_total-votes

In [ ]:
gz2_cols = [col for col in gz2_desi_matches.columns.values if 'gz2' in col] + ['dr8_id']
df = pd.merge(df, gz2_desi_matches[gz2_cols], how='left', on='dr8_id')

In [ ]:
df[label_metadata.gz2_ortho_label_cols] = df[label_metadata.gz2_ortho_label_cols].fillna(0)


In [ ]:
df['smooth-or-featured-gz2_smooth']

0          0.0
1         36.0
2         31.0
3          0.0
4         22.0
          ... 
415597     0.0
415598     0.0
415599     0.0
415600     0.0
415601     0.0
Name: smooth-or-featured-gz2_smooth, Length: 415602, dtype: float64

In [ ]:
df.to_parquet('data/gz3d_and_desi_master_catalog.parquet', index=False)  # now with gz2 also

In [ ]:
df.columns.values

array(['ra_manga', 'dec_manga', 'mangaid', 'iauname', 'ifudesignsize',
       'manga_tileid', 'nsa_id', 'explorer_link',
       'gz_total_classifications', 'gz_bar_votes', 'gz_spiral_votes',
       'sample', 'relative_gz3d_fits_loc', 'best_match', 'sky_separation',
       'ra_desi', 'dec_desi', 'dr8_id', 'mag_r', 'mag_g', 'mag_z',
       'est_dr5_pixscale', 'galahad_fits_loc', 'galahad_jpg_loc',
       'relative_desi_fits_loc', 'relative_desi_jpg_loc',
       'relative_segmap_json_loc', 'relative_spiral_mask_loc',
       'relative_bar_mask_loc', 'smooth-or-featured-dr12_smooth',
       'smooth-or-featured-dr12_featured-or-disk',
       'smooth-or-featured-dr12_artifact', 'disk-edge-on-dr12_yes',
       'disk-edge-on-dr12_no', 'has-spiral-arms-dr12_yes',
       'has-spiral-arms-dr12_no', 'bar-dr12_yes', 'bar-dr12_no',
       'bulge-size-dr12_dominant', 'bulge-size-dr12_obvious',
       'bulge-size-dr12_none', 'how-rounded-dr12_completely',
       'how-rounded-dr12_in-between', 'how-roun